# Question to be answered:

- Is the accuracy of model/human significantly better? In both force and mass questions?
- Is the distribution of responses significantly different?

In [1]:
import isaac.constants
isaac.constants.TQDM_DISABLE = True

from torch import nn
from torch.nn import Softmax
from isaac.utils import get_cuda_device_if_available
import joblib

from isaac.dataset import read_dataset, prepare_dataset
from isaac.models import MultiBranchModel, ComplexRNNModel
from isaac.constants import BASIC_TRAINING_COLS, MASS_CLASS_COLS, FORCE_CLASS_COLS, RTHETA_COLS, XY_RTHETA_COLS, XY_VXVY_RTHETA_COLS
from isaac.evaluation import predict_with_a_group_of_saved_models, evaluate_saved_model
from isaac.statistical_tests import z_test

import torch
import glob
from torch.autograd import Variable
import numpy as np
import pandas as pd
from tqdm import tqdm


In [2]:
INTERVAL_SIZE = 1
FPS = 60
STEP_SIZE = 3
PD_STEP_SIZE = 10
SEQ_END = 2700

In [3]:
device = get_cuda_device_if_available()
print(device)

cpu


In [4]:
normalise_data = True
scaler_path = "scalers/passive_rtheta_scaler.sk"
network_dims = (len(RTHETA_COLS), 25, 3, 0.5)
dataset_path = "../new_exp_data/exp7_passive_rtheta.h5"
class_columns = [list(MASS_CLASS_COLS), list(FORCE_CLASS_COLS)]
multiclass = True
DATASET = read_dataset(dataset_path)


def get_question_predictions_for_group_of_models(question_type):    
    models = sorted(glob.glob("models/rtheta/best_"+question_type+"_model_seed_*.pt"))

    group_predictions = []
    
    predictions = predict_with_a_group_of_saved_models(tqdm(models), network_dims, None, 
                                                       training_columns=RTHETA_COLS, 
                                                       class_columns=class_columns, step_size=STEP_SIZE, 
                                                       seq_end=SEQ_END, scaler_path=scaler_path,
                                                       arch=MultiBranchModel, multiclass=multiclass, trials=DATASET,
                                                       predict_seq2seq=True)

    predictions = torch.stack(predictions)
    
    print(predictions.shape)
    if question_type == "mass":
        predictions = predictions[:, :, :, 0]
    else:
        predictions = predictions[:, :, :, 1]

    return predictions

def get_question_accuracy_for_group_of_models(question_type):    
    model_paths = tqdm(sorted(glob.glob("models/rtheta/best_"+question_type+"_model_seed_*.pt")))

    accuracies, predicted = evaluate_saved_model(model_paths, network_dims, dataset_path, 
                                                 training_columns=RTHETA_COLS, class_columns=class_columns, 
                                                 step_size=STEP_SIZE, seq_end=SEQ_END, scaler_path=scaler_path,
                                                 arch=MultiBranchModel, multiclass=multiclass, trials=None)
    
    if question_type == "mass":
        question_index = 0
    else:
        question_index = 1

    accuracies = np.stack(accuracies)[:, question_index]
    
    predicted = [x[:, question_index].numpy() for x in predicted]

    return accuracies, predicted

# T-test for MASS questions

## Load model's predictions

In [5]:
print("MASS")
question_type = "mass"
group_mass_seq_prediction = get_question_predictions_for_group_of_models(question_type)

print("\nFORCE")
question_type = "force"
group_force_seq_prediction = get_question_predictions_for_group_of_models(question_type)

  0%|          | 0/25 [00:00<?, ?it/s]

MASS


/home/hector/miniconda3/envs/diss/lib/python3.5/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator StandardScaler from version 0.23.2 when using version 0.22.2.post1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
  0%|          | 0/25 [00:00<?, ?it/s]/home/hector/miniconda3/envs/diss/lib/python3.5/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator StandardScaler from version 0.23.2 when using version 0.22.2.post1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


torch.Size([25, 36, 45, 2, 3])

FORCE


100%|██████████| 25/25 [00:11<00:00,  2.16it/s]

torch.Size([25, 36, 45, 2, 3])


In [6]:
mass_solutions = [trial[list(MASS_CLASS_COLS)].idxmax(axis=1).unique()[0] for trial in DATASET]
force_solutions = [trial[list(FORCE_CLASS_COLS)].idxmax(axis=1).unique()[0] for trial in DATASET]

In [7]:
s = Softmax(dim=-1)
group_force_seq_prediction = s(group_force_seq_prediction)
group_mass_seq_prediction = s(group_mass_seq_prediction)

In [8]:
sim_second = [i for _ in range(25) for _ in range(36) for i in range(1, 46)]
model_seed = [i for i in range(25) for _ in range(36) for _ in range(1, 46)]
trial_number = [i for _ in range(25) for i in range(36) for _ in range(1, 46)]

In [9]:
mass_df = pd.DataFrame(data=group_mass_seq_prediction.reshape(40500, 3).numpy(), 
                       columns=["rnn_%s" % cl for cl in MASS_CLASS_COLS])
mass_df["sim_second"] = sim_second
mass_df["model_seed"] = model_seed
mass_df["trial_number"] = trial_number
mass_df["solution"] = [mass_solutions[trial_id] for trial_id in trial_number]

force_df = pd.DataFrame(data=group_force_seq_prediction.reshape(40500, 3).numpy(), 
                       columns=["rnn_%s" % cl for cl in FORCE_CLASS_COLS])
force_df["sim_second"] = sim_second
force_df["model_seed"] = model_seed
force_df["trial_number"] = trial_number
force_df["solution"] = [force_solutions[trial_id] for trial_id in trial_number]

In [10]:
import matplotlib.pyplot as plt
import seaborn as sns

sns.set(font_scale=3)
sns.set_style("white")

# Plot all Force probabilities throughout the trials

In [14]:
for trial_i in range(force_df["trial_number"].max() + 1):
    trial_df = force_df.query("trial_number == %d" % trial_i)
    
    plt.figure(figsize=(40, 6))
           
    ax = sns.lineplot(y="rnn_attract", x="sim_second", data=trial_df, linewidth=20, label="attract", markers=True)    
    ax = sns.lineplot(y="rnn_repel", x="sim_second", data=trial_df, ax=ax, linewidth=20, label="repel")    
    ax = sns.lineplot(y="rnn_none", x="sim_second", data=trial_df, ax=ax, linewidth=20, label="none")
    sns.despine(ax=ax)
    
    ax.set_ylabel("RNN probability", weight="bold")
    ax.set_yticks([0, 0.5, 1])
    ax.set_yticklabels(ax.get_yticks(), weight="bold")
    ax.set_xlim(0, 45)
    ax.set_xticklabels([])
    ax.set_xlabel("")
    ax.legend(loc=2, prop={'weight':'bold'})
    
    solution = trial_df.solution.unique()[0]
    if solution == "attract":
        ax.lines[1].set_linestyle("--")
        ax.lines[2].set_linestyle("--")
    elif solution == "repel":
        ax.lines[0].set_linestyle("--")
        ax.lines[2].set_linestyle("--")
    else:
        ax.lines[0].set_linestyle("--")
        ax.lines[1].set_linestyle("--")        
        
    plt.savefig("cogsci_images/RNN_all_prob_force_plot_%d.png" % trial_i, bbox_inches='tight')
    plt.close()

# Plot all Mass probabilities throughout the trials

In [ ]:
for trial_i in range(mass_df["trial_number"].max() + 1):
    trial_df = mass_df.query("trial_number == %d" % trial_i)
    plt.figure(figsize=(40, 6))
           
    ax = sns.lineplot(y="rnn_A", x="sim_second", data=trial_df, linewidth=20, label="A", markers=True)    
    ax = sns.lineplot(y="rnn_B", x="sim_second", data=trial_df, ax=ax, linewidth=20, label="B")    
    ax = sns.lineplot(y="rnn_same", x="sim_second", data=trial_df, ax=ax, linewidth=20, label="same")
    sns.despine(ax=ax)
    
    ax.set_ylabel("RNN probability", weight="bold")
    ax.set_yticks([0, 0.5, 1])
    ax.set_yticklabels(ax.get_yticks(), weight="bold")
    ax.set_xlim(0, 45)
    ax.set_xticklabels([])
    ax.set_xlabel("")
    ax.legend(loc=2, prop={'weight':'bold'})
    
    solution = trial_df.solution.unique()[0]
    if solution == "A":
        ax.lines[1].set_linestyle("--")
        ax.lines[2].set_linestyle("--")
    elif solution == "B":
        ax.lines[0].set_linestyle("--")
        ax.lines[2].set_linestyle("--")
    else:
        ax.lines[0].set_linestyle("--")
        ax.lines[1].set_linestyle("--")        
        
    plt.savefig("cogsci_images/RNN_all_prob_mass_plot_%d.png" % trial_i, bbox_inches='tight')
    plt.close()

## Load human results

In [11]:
from isaac.visualization import make_frame_curried
import moviepy.editor as mpy
from scipy import misc


def make_clip(trial_data, probability_path, entropy_path):
    probability_image = misc.imread(probability_path)
    probability_image = misc.imresize(probability_image, 25)
    
    entropy_image = misc.imread(entropy_path)
    entropy_image = misc.imresize(entropy_image, 25)
        
    duration = len(trial_data)

    n_bodies = sum(["o"+str(i)+".x" in list(trial_data.columns) for i in range(1, 5)])
    
    while (len(trial_data) + 1) % 60 != 0:
        trial_data = trial_data.append(trial_data.iloc[-1], ignore_index=True)
    make_frame = make_frame_curried(trial_data, n_bodies, probability_image, entropy_image)
    clip = mpy.VideoClip(make_frame, duration=duration / 60)
    return clip, trial_data

pygame 1.9.6
Hello from the pygame community. https://www.pygame.org/contribute.html


In [12]:
%load_ext autoreload
%autoreload 2

# Create videos for comparing evidence seen by RNN and by IO

In [ ]:
probability_paths = ["cogsci_images/RNN_all_prob_mass_plot_%d.png" % trial_i for trial_i in range(len(mass_df))]
entropy_paths = ["cogsci_images/PD_RNN_ent_mass_plot_%d.png" % trial_i for trial_i in range(len(mass_df))]
replays = read_dataset("../new_exp_data/exp7_passive.h5")

for trial_i, (prob_path, ent_path, replay) in enumerate(zip(probability_paths, entropy_paths, replays)):
    clip = make_clip(replay, prob_path, ent_path)[0]
    clip.write_videofile("cogsci_images/trial_%d_mass_video_and_prob_plot.mp4" % trial_i, fps=60)

In [15]:
probability_paths = ["cogsci_images/RNN_all_prob_force_plot_%d.png" % trial_i for trial_i in range(len(mass_df))]
entropy_paths = ["cogsci_images/PD_RNN_ent_force_plot_%d.png" % trial_i for trial_i in range(len(mass_df))]
replays = read_dataset("../new_exp_data/exp7_passive.h5")

for trial_i, (prob_path, ent_path, replay) in enumerate(zip(probability_paths, entropy_paths, replays)):
    clip = make_clip(replay, prob_path, ent_path)[0]
    clip.write_videofile("cogsci_images/trial_%d_force_video_and_prob_plot.mp4" % trial_i, fps=60)

[MoviePy] >>>> Building video cogsci_images/trial_0_force_video_and_prob_plot.mp4
[MoviePy] Writing video cogsci_images/trial_0_force_video_and_prob_plot.mp4


100%|█████████▉| 2700/2701 [00:30<00:00, 89.43it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: cogsci_images/trial_0_force_video_and_prob_plot.mp4 

[MoviePy] >>>> Building video cogsci_images/trial_1_force_video_and_prob_plot.mp4
[MoviePy] Writing video cogsci_images/trial_1_force_video_and_prob_plot.mp4


100%|█████████▉| 2700/2701 [00:30<00:00, 88.68it/s]

[MoviePy] Done.
[MoviePy] >>>> Video ready: cogsci_images/trial_1_force_video_and_prob_plot.mp4 



...

[MoviePy] Done.
[MoviePy] >>>> Video ready: cogsci_images/trial_33_force_video_and_prob_plot.mp4 

[MoviePy] >>>> Building video cogsci_images/trial_34_force_video_and_prob_plot.mp4
[MoviePy] Writing video cogsci_images/trial_34_force_video_and_prob_plot.mp4


100%|█████████▉| 2700/2701 [00:30<00:00, 89.46it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: cogsci_images/trial_34_force_video_and_prob_plot.mp4 

[MoviePy] >>>> Building video cogsci_images/trial_35_force_video_and_prob_plot.mp4
[MoviePy] Writing video cogsci_images/trial_35_force_video_and_prob_plot.mp4


100%|█████████▉| 2700/2701 [00:30<00:00, 88.82it/s]

[MoviePy] Done.
[MoviePy] >>>> Video ready: cogsci_images/trial_35_force_video_and_prob_plot.mp4 

